# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

> You can use the classwork notebook and also [my Everything scraping reference](https://jonathansoma.com/everything/scraping)

I shouldn't have to say this, but **do not use ChatGPT for this assignment.**

## Setup: Import what you'll need to scrape the page

We'll be using either Playwrightfor this, *not* requests.

In [33]:
from playwright.async_api import async_playwright

In [34]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [35]:
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

<Response url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' request=<Request url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' method='GET'>>

In [37]:
await page.get_by_label("Search by License Program Type").select_option('Cosmetologists')

['COS']

In [38]:
await page.get_by_label("Search by Last Name").fill("Nguyen")

In [42]:
await page.locator("input[name=\"B1\"]").click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't reaaallly tabular data (or there's just too much to clean up).

Once you've loaded the page: **what selector are you going to do use to find the data?** In class we used things like `.article` or `.head-list`.

> You honestly can do this all with Playwright, no BeautifulSoup involved! But we didn't cover that in class so you should stick to this method:
> 
> ```python
> html = await page.content()
> doc = BeautifulSoup(html)
> ```

In [45]:
from bs4 import BeautifulSoup

html = await page.content()
soup_doc = BeautifulSoup(html)
print(soup_doc.prettify())

<html class="supports csstransforms3d" lang="en">
 <head>
  <!-- Meta Tags -->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <!-- Favicon -->
  <link href="/images/favicon.png" rel="shortcut icon" type="image/x-icon"/>
  <!-- Stylesheets -->
  <link href="/css/reset.css" rel="stylesheet" type="text/css"/>
  <link href="/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <link href="/css/animate.css" rel="stylesheet" type="text/css"/>
  <link href="/css/main-stylesheet.css" rel="stylesheet" type="text/css"/>
  <link href="/css/lightbox.css" rel="stylesheet" type="text/css"/>
  <link href="/css/shortcodes.css" rel="stylesheet" type="text/css"/>
  <link href="/css/custom-fonts.css" rel="stylesheet" type="text/css"/>
  <link href="/css/custom-colors.css" rel="stylesheet" type="text/css"/>
  <link href="/css/responsive.css" rel="stylesheet" type="text/css"/>
  <lin

### Grab the rows, loop through each result and print the entire row

It's probably too many rows to do all at once: once you have a list, use `[:10]` to only show the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print("------")
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

In [58]:
table = soup_doc.find("table").find_all('tr')

for row in table[:10]:
    print("------")
    print(row.text)

------
Name and LocationOrderBasis for Order
------
 NGUYEN, ELLIE UYEN  City: HUMBLE County: HARRIS Zip Code: 77346 License #: 798636Complaint # COS20240003068 Date: 10/30/2024Respondents Ellie Uyen Nguyen and Bao Q. Truong are assessed an administrative penalty in the amount of $3,750. Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use.
------
 NGUYEN, PHUC  City: HOUSTON County: HARRIS Zip Code: 77083 License #: 807327Complaint # COS20230005653 Date: 1

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll probably get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – like `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [60]:
table = soup_doc.find("table").find_all('tr')

for row in table[:10]:
    try:
        print(row.find_all('td')[0].span.string)
    except:
        print("This one didn't work!")
        

This one didn't work!
NGUYEN, ELLIE UYEN 
NGUYEN, PHUC 
NGUYEN, TRONG 
NGUYEN, SA 
NGUYEN, LYNH 
NGUYEN, LISA 
NGUYEN, CHAU MINH 
NGUYEN, KEVIN C
NGUYEN, THANH D


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: Or I guess you could just skip the one with the problem using try/except...*

In [64]:
for row in table[:10]:
    try:
        print("-----")
        print(row.find_all('td')[2].string)
    except:
        print("This one didn't work!")
      

-----
This one didn't work!
-----
Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use.
-----
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required daily, bi-weekly, and before use by each patron.
-----
Respondents failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foo

## Loop through each result, printing the complaint number

Output should look similar to this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [66]:
for row in table[:10]:
    try:
        print("-----")
        print(row.find_all('td')[0].find_all('span')[-1].string)
    except:
        print("This one didn't work!")
      

-----
This one didn't work!
-----
COS20240003068
-----
COS20230005653
-----
COS20240011518
-----
COS20240008640
-----
COS20230004635
-----
COS20230010686
-----
COS20220007736
-----
COS20230020006
-----
COS20240000594


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> _**Tip:** Depending on how you do this, you might want to figure out how to search for "next siblings" or "following siblings"_

In [92]:
all_nguyens = []

for row in table:
    each_nguyen = {}
    try:
        name = row.find_all('td')[0].find_all('span')[0].string
        each_nguyen['name'] = name

        city = row.find_all('td')[0].find_all('span')[2].string
        each_nguyen['city'] = city

        county = row.find_all('td')[0].find_all('span')[4].string
        each_nguyen['county'] = county

        zip_code = row.find_all('td')[0].find_all('span')[6].string
        each_nguyen['zip code'] = zip_code

        complaint_no = row.find_all('td')[0].find_all('span')[-1].string
        each_nguyen['complaint_no'] = complaint_no

        license_numbers = row.find_all('td')[0].find_all('span')[-3].string
        each_nguyen['license_numbers'] = license_numbers
        
        violation_desc = row.find_all('td')[2].string
        each_nguyen['complaint'] = violation_desc

        all_nguyens.append(each_nguyen)
    
    except: 
        pass
        
print(all_nguyens)

[{'name': 'NGUYEN, ELLIE UYEN ', 'city': 'HUMBLE', 'county': 'HARRIS', 'zip code': '77346', 'complaint_no': 'COS20240003068', 'license_numbers': '798636', 'complaint': "Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use."}, {'name': 'NGUYEN, PHUC ', 'city': 'HOUSTON', 'county': 'HARRIS', 'zip code': '77083', 'complaint_no': 'COS20230005653', 'license_numbers': '807327', 'complaint': 'Respondent failed to follow whirlpool foot spas cleaning and sanitizatio

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [94]:
import pandas as pd

df = pd.DataFrame(all_nguyens)
df

,name,city,county,zip code,complaint_no,license_numbers,complaint
0,"NGUYEN, ELLIE UYEN",HUMBLE,HARRIS,77346,COS20240003068,798636,"Respondents failed to clean, disinfect, and st..."
1,"NGUYEN, PHUC",HOUSTON,HARRIS,77083,COS20230005653,807327,Respondent failed to follow whirlpool foot spa...
2,"NGUYEN, TRONG",FORT WORTH,TARRANT,76177,COS20240011518,808004,Respondents failed to follow whirlpool foot sp...
3,"NGUYEN, SA",AUSTIN,TRAVIS,78729,COS20240008640,799667,Respondent failed to follow whirlpool foot spa...
4,"NGUYEN, LYNH",RICHARDSON,DALLAS,75081,COS20230004635,811783,Respondent failed to follow whirlpool foot spa...
...,...,...,...,...,...,...,...
74,"NGUYEN, KELLY PHUONG N",GARDEN GROVE,OUT OF STATE,92843,COS20220006797,751553,Respondent failed to keep a record of the date...
75,"NGUYEN, QUOC BAO",ARLINGTON,TARRANT,76018,COS20210009267,775512,Respondent failed to follow whirlpool foot spa...
76,"NGUYEN, DANIEL THAO",PLANO,COLLIN,75093,COS20220004501,774875,Respondent failed to clean and sanitize whirlp...
77,"NGUYEN, THANH THU",EL PASO,EL PASO,79904,COS20210012742,728312,Respondent failed to clean and sanitize whirlp...


In [95]:
df.to_csv("output.csv")

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [ ]:
looks okay!